In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：使用TensorFlow提交一个超参数调优训练作业

安装

安装最新（预览）版本的Vertex SDK。

In [ ]:
! pip3 install -U google-cloud-aiplatform --user

安装Google的*云存储*库。

In [ ]:
! pip3 install google-cloud-storage

### 重新启动内核

一旦您已安装了Vertex SDK和Google *cloud-storage*，您需要重新启动笔记本内核以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*确保在GPU运行时下运行此笔记本（如果有这个选项）。在Colab中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的GCP项目

**无论您的笔记本环境如何，都需要执行以下步骤。**

1. [选择或创建 GCP 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度用于计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud笔记本中。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保 Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会运行以`!`为前缀的行作为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您也可以更改“REGION”变量，该变量用于笔记本的其余部分操作。 以下是Vertex AI支持的区域。 我们建议尽可能选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您不能在Vertex上使用多区域存储桶进行培训。 并非所有区域都支持所有Vertex服务。 有关每个区域的最新支持，请参见 [Vertex AI服务的区域支持](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行直播教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源上发生名称冲突，您会为每个实例会话创建一个时间戳，并将其附加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的GCP账户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过验证。请跳过此步骤。

*注意: 如果您在Vertex笔记本上运行单元格，单元格会自动跳过执行身份验证步骤。*

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### 创建一个云存储存储桶

**无论您使用的是笔记本环境如何，都需要按照以下步骤操作。**

本教程旨在使用存储在公共云存储桶中的训练数据以及本地云存储桶用于批量预测。您也可以使用您自己存储在本地云存储桶中的训练数据。

请在下方设置您的云存储桶的名称。这个名称必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时才能执行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

最后，通过检查云存储桶的内容来验证对其的访问权限：

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常数

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常数

#### 导入Vertex SDK

将Vertex SDK导入到我们的Python环境中。

In [ ]:
import os
import sys
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

常量用于Vertex AI设置：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的Vertex AI API服务端点。
- `API_PREDICT_ENDPOINT`：用于预测的Vertex AI API服务端点。
- `PARENT`：数据集、模型和端点资源的Vertex AI位置根路径。

In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

客户端

Vertex SDK 以客户端/服务器模型工作。在您的一侧（Python 脚本）中，您将创建一个客户端，向服务器（Vertex）发送请求并接收响应。

在本教程中，您将使用多个客户端，因此请提前设置它们所有。

- 用于受管数据集的数据集服务。
- 用于受管模型的模型服务。
- 用于训练的管道服务。
- 用于部署的端点服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于提供预测的预测服务。*注意*：预测具有不同的服务端点。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

准备一个训练脚本

### 包裹组装

In [ ]:
# Make folder for python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\
tag_build =\n\
tag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\
# Requires TensorFlow Datasets\n\
setuptools.setup(\n\
    install_requires=[\n\
        'tensorflow_datasets==1.3.0',\n\
    ],\n\
    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\
Name: Hyperparameter Tuning - Boston Housing\n\
Version: 0.0.0\n\
Summary: Demonstration hyperparameter tuning script\n\
Home-page: www.google.com\n\
Author: Google\n\
Author-email: aferlitsch@gmail.com\n\
License: Public\n\
Description: Demo\n\
Platform: Vertex AI"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### Task.py 的内容

In [ ]:
%%writefile custom/trainer/task.py
# hyperparameter tuningfor Boston Housing
  
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default='/tmp/saved_model', type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--units', dest='units',
                    default=64, type=int,
                    help='Number of units.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

def make_dataset():
  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)
    
  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)

# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(args.units, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(args.units, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

model = build_and_compile_dnn_model()

# Instantiate the HyperTune reporting object
hpt = HyperTune()

# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_loss',
        metric_value=logs['val_loss'],
        global_step=epoch)

# Train the model
BATCH_SIZE = 16
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=BATCH_SIZE, validation_split=0.1, callbacks=[HPTCallback()])
model.save(args.model_dir)


### 将培训脚本存储在您的云存储桶中

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz gs://$BUCKET_NAME/hpt_boston_housing.tar.gz

训练一个模型

### [projects.locations.hyperparameterTuningJob.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.hyperparameterTuningJobs/create)

### [projects.locations.hyperparameterTuningJob.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.hyperparameterTuningJobs/create)

#### 请求

In [ ]:
JOB_NAME = "hyperparameter_tuning_" + TIMESTAMP

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": {"machine_type": "n1-standard-4", "accelerator_count": 0},
        "python_package_spec": {
            "executor_image_uri": "gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest",
            "package_uris": ["gs://" + BUCKET_NAME + "/hpt_boston_housing.tar.gz"],
            "python_module": "trainer.task",
            "args": ["--model-dir=" + "gs://{}/{}".format(BUCKET_NAME, JOB_NAME)],
        },
    }
]

STUDY_SPEC = {
    "metrics": [
        {"metric_id": "val_loss", "goal": aip.StudySpec.MetricSpec.GoalType.MINIMIZE}
    ],
    "parameters": [
        {
            "parameter_id": "lr",
            "discrete_value_spec": {"values": [0.001, 0.01, 0.1]},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        },
        {
            "parameter_id": "units",
            "integer_value_spec": {"min_value": 32, "max_value": 256},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        },
    ],
    "algorithm": aip.StudySpec.Algorithm.RANDOM_SEARCH,
}

hyperparameter_tuning_job = aip.HyperparameterTuningJob(
    display_name=JOB_NAME,
    trial_job_spec={"worker_pool_specs": WORKER_POOL_SPEC},
    study_spec=STUDY_SPEC,
    max_trial_count=6,
    parallel_trial_count=1,
)

print(
    MessageToJson(
        aip.CreateHyperparameterTuningJobRequest(
            parent=PARENT, hyperparameter_tuning_job=hyperparameter_tuning_job
        ).__dict__["_pb"]
    )
)

{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "hyperparameterTuningJob": {
    "displayName": "hyperparameter_tuning_20210226020029",
    "studySpec": {
      "metrics": [
        {
          "metricId": "val_loss",
          "goal": "MINIMIZE"
        }
      ],
      "parameters": [
        {
          "parameterId": "lr",
          "discreteValueSpec": {
            "values": [
              0.001,
              0.01,
              0.1
            ]
          },
          "scaleType": "UNIT_LINEAR_SCALE"
        },
        {
          "parameterId": "units",
          "integerValueSpec": {
            "minValue": "32",
            "maxValue": "256"
          },
          "scaleType": "UNIT_LINEAR_SCALE"
        }
      ],
      "algorithm": "RANDOM_SEARCH"
    },
    "maxTrialCount": 6,
    "parallelTrialCount": 1,
    "trialJobSpec": {
      "workerPoolSpecs": [
        {
          "machineSpec": {
            "machineType": "n1-standard-4"
          },
          "replicaCount": "1",
          "pythonPackageSpec": {
            "executorImageUri": "gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest",
            "packageUris": [
              "gs://migration-ucaip-trainingaip-20210226020029/hpt_boston_housing.tar.gz"
            ],
            "pythonModule": "trainer.task",
            "args": [
              "--model-dir=gs://migration-ucaip-trainingaip-20210226020029/hyperparameter_tuning_20210226020029"
            ]
          }
        }
      ]
    }
  }
}

#### 呼叫

In [ ]:
request = clients["job"].create_hyperparameter_tuning_job(
    parent=PARENT, hyperparameter_tuning_job=hyperparameter_tuning_job
)

回复

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

{
  "名称": "projects/116273516712/locations/us-central1/hyperparameterTuningJobs/5264408897233354752",
  "显示名称": "hyperparameter_tuning_20210226020029",
  "studySpec": {
    "指标": [
      {
        "指标ID": "val_loss",
        "目标": "最小化"
      }
    ],
    "参数": [
      {
        "参数ID": "lr",
        "离散值规范": {
          "值": [
            0.001,
            0.01,
            0.1
          ]
        },
        "比例类型": "单位线性比例"
      },
      {
        "参数ID": "units",
        "整数值规范": {
          "最小值": "32",
          "最大值": "256"
        },
        "比例类型": "单位线性比例"
      }
    ],
    "算法": "随机搜索"
  },
  "最大试验次数": 6,
  "并行试验次数": 1,
  "试验作业规范": {
    "工作池规范": [
      {
        "机器规范": {
          "机器类型": "n1-standard-4"
        },
        "副本数量": "1",
        "磁盘规范": {
          "启动磁盘类型": "pd-ssd",
          "启动磁盘大小GB": 100
        },
        "Python包规范": {
          "执行器映像URI": "gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest",
          "软件包URIs": [
            "gs://migration-ucaip-trainingaip-20210226020029/hpt_boston_housing.tar.gz"
          ],
          "Python模块": "trainer.task",
          "参数": [
            "--model-dir=gs://migration-ucaip-trainingaip-20210226020029/hyperparameter_tuning_20210226020029"
          ]
        }
      }
    ]
  },
  "状态": "作业状态待定",
  "创建时间": "2021-02-26T02:02:02.787187Z",
  "更新时间": "2021-02-26T02:02:02.787187Z"
}
```

In [ ]:
# The full unique ID for the hyperparameter tuningjob
hyperparameter_tuning_id = request.name
# The short numeric ID for the hyperparameter tuningjob
hyperparameter_tuning_short_id = hyperparameter_tuning_id.split("/")[-1]

print(hyperparameter_tuning_id)

###[projects.locations.hyperparameterTuningJob.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.hyperparameterTuningJobs/get)

将上述英文文本翻译为中文：[项目.位置.超参数调整作业.获取](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.hyperparameterTuningJobs/get)

#### 电话

In [ ]:
request = clients["job"].get_hyperparameter_tuning_job(name=hyperparameter_tuning_id)

回复

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*示例输出*：
```
{
  "name": "projects/116273516712/locations/us-central1/hyperparameterTuningJobs/5264408897233354752",
  "displayName": "hyperparameter_tuning_20210226020029",
  "studySpec": {
    "metrics": [
      {
        "metricId": "val_loss",
        "goal": "MINIMIZE"
      }
    ],
    "parameters": [
      {
        "parameterId": "lr",
        "discreteValueSpec": {
          "values": [
            0.001,
            0.01,
            0.1
          ]
        },
        "scaleType": "UNIT_LINEAR_SCALE"
      },
      {
        "parameterId": "units",
        "integerValueSpec": {
          "minValue": "32",
          "maxValue": "256"
        },
        "scaleType": "UNIT_LINEAR_SCALE"
      }
    ],
    "algorithm": "RANDOM_SEARCH"
  },
  "maxTrialCount": 6,
  "parallelTrialCount": 1,
  "trialJobSpec": {
    "workerPoolSpecs": [
      {
        "machineSpec": {
          "machineType": "n1-standard-4"
        },
        "replicaCount": "1",
        "diskSpec": {
          "bootDiskType": "pd-ssd",
          "bootDiskSizeGb": 100
        },
        "pythonPackageSpec": {
          "executorImageUri": "gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest",
          "packageUris": [
            "gs://migration-ucaip-trainingaip-20210226020029/hpt_boston_housing.tar.gz"
          ],
          "pythonModule": "trainer.task",
          "args": [
            "--model-dir=gs://migration-ucaip-trainingaip-20210226020029/hyperparameter_tuning_20210226020029"
          ]
        }
      }
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-02-26T02:02:02.787187Z",
  "updateTime": "2021-02-26T02:02:02.787187Z"
}
```

## 等待研究完成

In [ ]:
while True:
    response = clients["job"].get_hyperparameter_tuning_job(
        name=hyperparameter_tuning_id
    )
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Study trials have not completed:", response.state)
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        print("Study trials have completed:", response.end_time - response.start_time)
        break
    time.sleep(20)

## 请审阅研究结果

In [ ]:
best = (None, None, None, 0.0)
response = clients["job"].get_hyperparameter_tuning_job(name=hyperparameter_tuning_id)
for trial in response.trials:
    print(MessageToJson(trial.__dict__["_pb"]))
    # Keep track of the best outcome
    try:
        if float(trial.final_measurement.metrics[0].value) > best[3]:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
    except:
        pass

print()
print("ID", best[0])
print("Decay", best[1])
print("Learning Rate", best[2])
print("Validation Accuracy", best[3])

{
  "id": "1",
  "state": "成功",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.1
    },
    {
      "parameterId": "units",
      "value": 80.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "19",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 46.61515110294993
      }
    ]
  },
  "startTime": "2021-02-26T02:05:16.935353384Z",
  "endTime": "2021-02-26T02:12:44Z"
}
{
  "id": "2",
  "state": "成功",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.01
    },
    {
      "parameterId": "units",
      "value": 45.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "19",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 32.55313952376203
      }
    ]
  },
  "startTime": "2021-02-26T02:15:31.357856840Z",
  "endTime": "2021-02-26T02:24:18Z"
}
{
  "id": "3",
  "state": "成功",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.1
    },
    {
      "parameterId": "units",
      "value": 70.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "19",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 42.709188321741614
      }
    ]
  },
  "startTime": "2021-02-26T02:26:40.704476222Z",
  "endTime": "2021-02-26T02:34:21Z"
}
{
  "id": "4",
  "state": "成功",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.01
    },
    {
      "parameterId": "units",
      "value": 173.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "17",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 46.12480219399057
      }
    ]
  },
  "startTime": "2021-02-26T02:37:45.275581053Z",
  "endTime": "2021-02-26T02:51:07Z"
}
{
  "id": "5",
  "state": "成功",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.01
    },
    {
      "parameterId": "units",
      "value": 223.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "19",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 24.875632611716664
      }
    ]
  },
  "startTime": "2021-02-26T02:53:32.612612421Z",
  "endTime": "2021-02-26T02:54:19Z"
}
{
  "id": "6",
  "state": "成功",
  "parameters": [
    {
      "parameterId": "lr",
      "value": 0.1
    },
    {
      "parameterId": "units",
      "value": 123.0
    }
  ],
  "finalMeasurement": {
    "stepCount": "13",
    "metrics": [
      {
        "metricId": "val_loss",
        "value": 43.352300690441595
      }
    ]
  },
  "startTime": "2021-02-26T02:56:47.323707459Z",
  "endTime": "2021-02-26T03:03:49Z"
}

ID 1
衰减率 0.1
学习率 80.0
验证准确度 46.61515110294993
}

清理

要清理此项目中使用的所有GCP资源，您可以[删除用于本教程的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_hpt_job = True
delete_bucket = True

# Delete the hyperparameter tuningusing the Vertex AI fully qualified identifier for the custome training
try:
    if delete_hpt_job:
        clients["job"].delete_hyperparameter_tuning_job(name=hyperparameter_tuning_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME